## Imports

In [12]:
import os
import json
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torch
import torchvision
from torchvision.io import read_image
from torchvision.transforms import ToTensor
from PIL import Image

## File paths

In [2]:
output_path = 'output'
img_folder = 'eccv_18_all_images_sm'

cis_test_ann_path = 'eccv_18_annotation_files/cis_test_annotations.json'
cis_val_ann_path = 'eccv_18_annotation_files/cis_val_annotations.json'
train_ann_path = 'eccv_18_annotation_files/train_annotations.json'
trans_test_ann_path = 'eccv_18_annotation_files/trans_test_annotations.json'
trans_val_ann_path = 'eccv_18_annotation_files/trans_val_annotations.json'

## Basic data exploration

In [3]:
cis_test_ann = json.load(open(cis_test_ann_path))
cis_val_ann = json.load(open(cis_val_ann_path))
train_ann = json.load(open(train_ann_path))
trans_test_ann = json.load(open(trans_test_ann_path))
trans_val_ann = json.load(open(trans_val_ann_path))

In [4]:
print('cis test set length:', len(cis_test_ann['images']))
print('cis val set length:', len(cis_val_ann['images']))
print('train set length:', len(train_ann['images']))
print('trans test set length:', len(trans_test_ann['images']))
print('trans val set length:', len(trans_val_ann['images']))

cis test set length: 15827
cis val set length: 3484
train set length: 13553
trans test set length: 23275
trans val set length: 1725


In [33]:
train_index = random.randint(0, len(train_ann['images'])-1)

In [ ]:
index = np.random.choice(range(len(train_ann['images'])), 1000)
images = [train_ann['images'][i] for i in index]

In [ ]:
trans_test_ann.keys()

In [ ]:
trans_test_ann['info']

In [ ]:
train_ann['images'][0]

In [ ]:
trans_val_ann['images'][1]

In [ ]:
train_ann['images'][2]

In [ ]:
cis_test_ann['images'][0]

In [ ]:
cis_test_ann['images'][1]

## Horizontal flip debugging

In [35]:
index = np.random.choice(range(len(train_ann['images'])), 1000)
images = [train_ann['images'][i] for i in index]

In [ ]:
i = 3
print(trans_test_ann['images'][i])
img_path = os.path.join('eccv_18_all_images_sm', trans_test_ann['images'][i]['file_name']) # to change

image = read_image(img_path)

fig, ax = plt.subplots()
ax.imshow(image[0].squeeze(), cmap='gray')

In [ ]:
i = 0

boxes = [trans_test_ann['annotations'][j]['bbox'] for j in range(len(trans_test_ann['annotations'])) 
         if trans_test_ann['annotations'][j]['image_id']==trans_test_ann['images'][i]['id'] 
         and 'bbox' in trans_test_ann['annotations'][j].keys()]

img_path = os.path.join('eccv_18_all_images_sm', trans_test_ann['images'][i]['file_name']) # to change

image = read_image(img_path)

fig, ax = plt.subplots()
ax.imshow(image[0].squeeze(), cmap='gray')

scale_x = image.shape[2] / trans_test_ann['images'][i]['width'] 
scale_y = image.shape[1] / trans_test_ann['images'][i]['height']

boxes = torch.as_tensor(boxes)

for i in range(boxes.shape[0]):
    boxes[i][0] = torch.round(boxes[i][0] * scale_x)
    boxes[i][1] = torch.round(boxes[i][1] * scale_y)
    boxes[i][2] = torch.round(boxes[i][2] * scale_x)
    boxes[i][3] = torch.round(boxes[i][3] * scale_y)

    boxes[i][2] = boxes[i][0] + boxes[i][2]
    boxes[i][3] = boxes[i][1] + boxes[i][3]

target = {}
target["boxes"] = boxes

rect = patches.Rectangle((boxes[0][0], boxes[0][1]), boxes[0][2]-boxes[0][0], 
                         boxes[0][3]-boxes[0][1], linewidth=2, edgecolor='r', facecolor='none')
ax.add_patch(rect)

In [ ]:
img_path

In [ ]:
image2 = Image.open(img_path).convert("RGB")

In [ ]:
image2

In [ ]:
image2.size

In [ ]:
conv = torchvision.transforms.ToTensor()

In [ ]:
width, height = image2.size[0], image2.size[1]

In [ ]:
print('width:', width)
print('height:', height)

In [ ]:
image_new = conv(image2)

In [ ]:
print('image_new.shape:', image_new.shape)

## Utils part

In [ ]:
# In paper :  ' ... and employ horizontal flipping for data augmentation. ( for detection)

import transforms as T   # from local files (from github repo)

data_transform = {'train': T.RandomHorizontalFlip(0.5)}

In [ ]:
# Returns a list with the idx of images with at least one bounding box (img_wbbox) and a 
# list with the number of bbox for each valid image (num_bbox)
def get_img_with_bbox(file_path):
  
    file = json.load(open(file_path))
    img_wbbox = []
    num_bbox = []

    for i in range(len(file['images'])):
        bboxes = [file['annotations'][j]['bbox'] 
                  for j in range(len(file['annotations'])) 
                  if file['annotations'][j]['image_id']==file['images'][i]['id'] 
                  and 'bbox' in file['annotations'][j].keys()]

        if len(bboxes)!=0:
            img_wbbox.append(i)

            num_bbox.append(len(bboxes))

    return img_wbbox, num_bbox

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, label_path, img_dir, valid_img, transform = None, target_transform=None):
        self.label_file = json.load(open(label_path))
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.valid_img = valid_img

    def __len__(self):
        return len(self.valid_img)

    def __getitem__(self, idx):
        
        idx = self.valid_img[idx] # consider only images with bbox annotations
        img_path = os.path.join(self.img_dir, self.label_file['images'][idx]['file_name'])
        image = read_image(img_path)

        conv = torchvision.transforms.ToTensor()
        # if image.shape[0]==1:
        # some images have only one channel, we convert them to rgb
        image = Image.open(img_path).convert("RGB")
        image = conv(image)

        boxes = [self.label_file['annotations'][j]['bbox'] 
                 for j in range(len(self.label_file['annotations'])) 
                 if self.label_file['annotations'][j]['image_id']==self.label_file['images'][idx]['id']]
        
        label = [self.label_file['annotations'][j]['category_id'] 
                 for j in range(len(self.label_file['annotations'])) 
                 if self.label_file['annotations'][j]['image_id']==self.label_file['images'][idx]['id']]

        # transform bbox coords to adjust for resizing
        scale_x = image.shape[2] / self.label_file['images'][idx]['width'] 
        scale_y = image.shape[1] / self.label_file['images'][idx]['height']

        boxes = torch.as_tensor(boxes)
        for i in range(boxes.shape[0]):
            boxes[i][0] = torch.round(boxes[i][0] * scale_x)
            boxes[i][1] = torch.round(boxes[i][1] * scale_y)
            boxes[i][2] = torch.round(boxes[i][2] * scale_x)
            boxes[i][3] = torch.round(boxes[i][3] * scale_y)

            boxes[i][2] = boxes[i][0] + boxes[i][2] # to transform to pytorch bbox format
            boxes[i][3] = boxes[i][1] + boxes[i][3]

            #boxes[i][0]*=scale_x
            #boxes[i][1]*=scale_y
            #boxes[i][2]*=scale_x
            #boxes[i][3]*=scale_y

        label=torch.as_tensor(label)
        label=torch.where(label==30,0,1)  # 0 if empty (categ id = 30), 1 if animal
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = label
        target["image_id"] = image_id
        target['area']=area
        target['iscrowd']=iscrowd

        # TO DO : resize all to same size

        if self.transform:
            # transform image AND target
            image, target = self.transform(image, target)
        if self.target_transform:
            label = self.target_transform(label)

        return image, target

In [ ]:
#### Some helper functions ####

import datetime
from datetime import datetime
from datetime import date

def date_to_integer(dt_time):
    '''
    importing datetime from image and converting to int
    arg@ datatime in format str "yyyy/mm/dd hh:mm:ss.ss"
    return: int@ yyyymmdd
    '''
    strDate = dt_time.replace("-", "/")
    date_object = datetime.strptime(strDate, "%Y/%m/%d %H:%M:%S")
    return 10000*date_object.year + 100*date_object.month + date_object.day

def getYear(dt_time):
    '''
    extract the year from image infromation
    return: year = int
    '''
    strDate = dt_time.replace("-", "/")
    date_object = datetime.strptime(strDate, "%Y/%m/%d %H:%M:%S")
    return date_object.year

def getMonth(dt_time):
    '''
    extract the month from image infromation
    return: year = int
    '''
    strDate = dt_time.replace("-", "/")
    date_object = datetime.strptime(strDate, "%Y/%m/%d %H:%M:%S")
    return date_object.month

def get_image_RGB(filename):
    '''
    # read an image form the dataset in : '/content/drive/MyDrive/AdvancedProjectML/data/eccv_18_all_images_sm'
    '''
    allImagesPath = '/content/drive/MyDrive/AdvancedProjectML/data/eccv_18_all_images_sm'
    img_path = os.path.join(allImagesPath,filename) 
    image = Image.open(img_path).convert("RGB")
    return image


def get_image_tensor(filename):
    '''
    # read an image form the dataset in : '/content/drive/MyDrive/AdvancedProjectML/data/eccv_18_all_images_sm'
    '''
    allImagesPath = '/content/drive/MyDrive/AdvancedProjectML/data/eccv_18_all_images_sm'
    img_path = os.path.join(allImagesPath,filename) 
    image = Image.open(img_path).convert("RGB")
    conv=torchvision.transforms.ToTensor()
    imageTensor=conv(image)
    return  imageTensor

## Saving as jpge ###
def saveImage(image, path, thisImageFileName):
    '''
    Saving as jpge.
    arg@ timage : Tensor
    arg@ thisImageFileName : str 
    arg @ path : str. The project path : '/content/drive/MyDrive/AdvancedProjectML/data'

    self.quality : int between 1 - 100. Define the image quality at save time 
    Ex from source: write_jpeg(input: torch.Tensor, filename: str, quality: int(1-100) = 75). It works with int = 100
    '''
    quality = 100
    if path == None:
        path = '/content/drive/MyDrive/AdvancedProjectML/data'
    if thisImageFileName == None:
        thisImageFileName = 'first'
    destinyPath = os.path.join(path, thisImageFileName)
    print("Saving...")
    write_jpeg(image, destinyPath, quality)

def showImageRGB(image): 
    '''
    input@ image filename
    e.i. fileName =  trans_val_ann['images'][i]['file_name'] 
    '''
    conv=torchvision.transforms.ToTensor()
    image_new=conv(image)
    plt.rcParams['font.size'] = 2
    fig, ax = plt.subplots(1,3,figsize=(8,4), dpi=150)
    ax[0].imshow(image_new[0], cmap='BuGn_r') #,cmap="gray"
    ax[1].imshow(image_new[1], cmap='BuGn_r')
    ax[2].imshow(image_new[2], cmap='BuGn_r')

def showImageTensor(image): 
    '''
    input@ image filename
    e.i. fileName =  trans_val_ann['images'][i]['file_name'] 
    '''
    plt.rcParams['font.size'] = 2
    fig, ax = plt.subplots(1,3,figsize=(6,2), dpi=150)
    ax[0].imshow(image[0]) #,cmap="gray"
    ax[1].imshow(image[1])
    ax[2].imshow(image[2])


### Import list of images names from a file
def importImageNamesListFromFile(finalPath):
    '''
    Import the names of all files in the directory as a list of str
    To Adapt: the directory path after tha main path.
    main path for this project: '/content/drive/MyDrive/AdvancedProjectML/data/....
    ex: finalPath = 'Background'
    '''
    mainPath = '/content/drive/MyDrive/AdvancedProjectML/data/'
    destinyPath = os.path.join(mainPath, finalPath)
    onlyFilesNames = [f for f in listdir(destinyPath)]
    return onlyFilesNames

def get_bbox(dataset,idx):
    '''
    retur@ list@ bbox from the image with filename=filename
    '''
    print(idx)
    return dataset['annotations'][idx]['bbox']




## Exemple of use

In [ ]:
# get the images bounding boxes *takes about 25sec*
train_valid_img,_ = get_img_with_bbox(train_ann_path)
cis_val_valid_img,_ = get_img_with_bbox(cis_val_ann_path)

In [ ]:
training_data = CustomImageDataset(train_ann_path, img_folder, train_valid_img)
valid_data = CustomImageDataset(cis_val_ann_path, img_folder, cis_val_valid_img)

train_dataloader = DataLoader(training_data, batch_size=1, shuffle=True, collate_fn=utils.collate_fn)

# In paper : ' We use a batch size of 1'

In [ ]:
train_features, train_labels = next(iter(train_dataloader))
#print(f"Feature batch shape: {train_features.size()}")
print(f"Target (Bbox) batch shape: {train_labels[0]['boxes'].size()}")
print(f"Target (category) batch shape: {train_labels[0]['labels'].size()}")

img = train_features[0][0].squeeze()
label = train_labels[0]['labels']
label_categ='animal'

if label[0]==0:
    label_categ='background'

fig, ax = plt.subplots()
ax.imshow(img,cmap="gray")
rect = patches.Rectangle((train_labels[0]['boxes'][0][0], train_labels[0]['boxes'][0][1]), train_labels[0]['boxes'][0][2]-train_labels[0]['boxes'][0][0], train_labels[0]['boxes'][0][3]-train_labels[0]['boxes'][0][1], linewidth=2, edgecolor='r', facecolor='none')
ax.add_patch(rect)
print(f"Label: {label_categ}")

In [ ]:
trans = data_transform['train']
img2, target2 = trans(image, target)

fig, ax = plt.subplots()
ax.imshow(img2, cmap="gray")

rect = patches.Rectangle((target2['boxes'][0][0], target2['boxes'][0][1]), 
                         target2['boxes'][0][2] - target2['boxes'][0][0], 
                         target2['boxes'][0][3] - target2['boxes'][0][1], 
                         linewidth=2, edgecolor='r', facecolor='none')
ax.add_patch(rect)